# Permeable Surface

This indicator was made for the 2022 base blueprint.

This notebook takes the NHD Pluse v 2.1 medium resolution catchments and calculates the mean value of the NLCD 2019 impervious surface per catchment.

Created by Amy Keister, last run by Amy Keister on 123 February, 2022. It took 53 minutes and 44 seconds to run. 

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace= r"E:\WORKING\BaseBlueprint\2022\3_Indicators\PermeableSurface\PermeableSurface.gdb"

In [4]:
# define final indicator outputs
SEout= r"E:\WORKING\BaseBlueprint\2022\3_Indicators\PermeableSurface\PermeableSurface.tif"

In [5]:
# define sub-indicator outputs to help with user support

In [11]:
# define rasters used for cell size, extent, and snapping
SEraster= r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\BaseBlueprintExtent2022.tif"

In [12]:
# define inputs
catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlusV21\NHDPlusV21_Catchment.gdb\CatchmentSECAS"
imp= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_impervious_2019_release_L48_all_files_20210604\nlcd_2019_impervious_l48_20210604\nlcd_2019_impervious_l48_20210604.img"

### Start Analysis

In [13]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [14]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

E:\WORKING\BaseBlueprint\2022\3_Indicators\PermeableSurface\PermeableSurface.gdb


### Zonal Statistics


In [15]:
# the NLCD impervous surfaces uses a value of 127 as NODATA. that is throwing off our calculations on the Mexico/US
# border. I need to change that 127 to NODATA before the zonal statistics
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con(imp, imp, None, "Value < 101"); out_raster.save("imp2")


In [16]:
# calculate mean impervious surface per catchment
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ZonalStatistics(catch, "GRIDCODE", "imp2", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMean")

In [17]:
# need to take 100 - the percent impervious value so it becomes percent permeable
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMean"); out_raster.save("permeable")

In [18]:
# reclassify permeable surface to create binned version

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)

out_raster = arcpy.sa.Reclassify("permeable", "VALUE", "0 70 1;70 90 2;90 95 3;95 100 4", "DATA"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [19]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 4:
        return '4 = >95% of catchment permeable (likely high water quality and supporting most sensitive aquatic species)'
    elif value == 3:
        return '3 = >90-95% of catchment permeable (likely declining water quality and supporting most aquatic species)'
    elif value == 2:
        return '2 = >70-90% of catchment permeable (likely degraded water quality and not supporting many aquatic species)'
    elif value == 1:
        return '1 = <=70% of catchment permeable (likely degraded instream flow, water quality, and aquatic species communities)'   
"""

In [20]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\3_Indicators\\PermeableSurface\\PermeableSurface.gdb\\indicator'>

In [21]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 4:
		return 0
	if Value == 3:
		return 97
	if Value == 2:
		return 151
	if Value == 1:
		return 204
	else:
		return 255
		
def Reclass2(Value):
	if Value == 4:
		return 0
	if Value == 3:
		return 69
	if Value == 2:
		return 131
	if Value == 1:
		return 204
	else:
		return 255
		
def Reclass3(Value):
	if Value == 4:
		return 224
	if Value == 3:
		return 237
	if Value == 2:
		return 247
	if Value == 1:
		return 255
	else:
		return 255
"""

In [22]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\3_Indicators\\PermeableSurface\\PermeableSurface.gdb\\indicator'>

In [23]:
# clip to SE 2021 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("indicator", SEraster); out_raster.save("SEMask")

In [24]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("SEMask", SEout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [25]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

3225.4730026721954
